In [253]:
# Library
import os
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels
import statsmodels.api as sm
import lightgbm as lgb
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_pinball_loss
from sklearn.feature_selection import RFE
from sklearn.preprocessing import FunctionTransformer
from sklego.preprocessing import RepeatingBasisFunction
from tqdm.auto import tqdm

import warnings
warnings.simplefilter('ignore')

In [254]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None
seed = 42

In [255]:
train_df = pd.read_csv('../data/train_data.csv')
test_df = pd.read_csv('../data/test_data.csv')
submission_df = pd.read_csv('../data/submission.csv')

In [256]:
def preprocessing(df, mode='train'):
    df_tmp = df.copy()
    input_year = 2021

    df_tmp['time'] = pd.to_datetime(df_tmp.date, format='%m/%d')
    df_tmp['year'] = df_tmp['time'].dt.year
    df_tmp['month'] = df_tmp['time'].dt.month
    df_tmp['day'] = df_tmp['time'].dt.day
    
    if mode=='train':
        df_tmp.loc[df_tmp['month']>3, 'year'] = input_year
        df_tmp.loc[df_tmp['month']<=3, 'year'] = input_year + 1
    else:
        df_tmp['year'] = input_year + 1
    df_tmp['time'] = pd.to_datetime({'year':df_tmp.year, 'month':df_tmp.month, 'day':df_tmp.day})
    df_tmp['weekday'] = df_tmp['time'].dt.weekday
    df_tmp['day_of_year'] = df_tmp['time'].dt.day_of_year
    return df_tmp

train_df = preprocessing(train_df, mode='train')
test_df = preprocessing(test_df, mode='test')

In [257]:
train_df.columns

Index(['id', 'date', 'highest', 'lowest', 'rain', 'ice1', 'ice2', 'ice3',
       'oden1', 'oden2', 'oden3', 'oden4', 'hot1', 'hot2', 'hot3', 'dessert1',
       'dessert2', 'dessert3', 'dessert4', 'dessert5', 'drink1', 'drink2',
       'drink3', 'drink4', 'drink5', 'drink6', 'alcol1', 'alcol2', 'alcol3',
       'snack1', 'snack2', 'snack3', 'bento1', 'bento2', 'bento3', 'bento4',
       'tild1', 'tild2', 'men1', 'men2', 'men3', 'men4', 'men5', 'men6',
       'time', 'year', 'month', 'day', 'weekday', 'day_of_year'],
      dtype='object')

In [258]:
target_columns = ['ice1', 'ice2', 'ice3',
    'oden1', 'oden2', 'oden3', 'oden4', 'hot1', 'hot2', 'hot3', 'dessert1',
    'dessert2', 'dessert3', 'dessert4', 'dessert5', 'drink1', 'drink2',
    'drink3', 'drink4', 'drink5', 'drink6', 'alcol1', 'alcol2', 'alcol3',
    'snack1', 'snack2', 'snack3', 'bento1', 'bento2', 'bento3', 'bento4',
    'tild1', 'tild2', 'men1', 'men2', 'men3', 'men4', 'men5', 'men6']

In [259]:
all_df = pd.concat([train_df, test_df], axis=0, ignore_index=True)

In [260]:
# # sin,cos変換を適用
# def sin_transformer(period):
#     return FunctionTransformer(lambda x: np.sin(x / period * 2 * np.pi))

# def cos_transformer(period):
#     return FunctionTransformer(lambda x: np.cos(x / period * 2 * np.pi))

# all_df_sincos = all_df.copy()
# all_df_sincos['day_of_year'] = all_df.time.dt.day_of_year

# # month -> sin, cos
# all_df_sincos['month_sin'] = sin_transformer(12).fit_transform(all_df_sincos['month'])
# all_df_sincos['month_cos'] = cos_transformer(12).fit_transform(all_df_sincos['month'])

# # week -> sin, cos
# all_df_sincos['weekday_sin'] = sin_transformer(7).fit_transform(all_df_sincos['weekday'])
# all_df_sincos['weekday_cos'] = cos_transformer(7).fit_transform(all_df_sincos['weekday'])

# # day -> sin, cos
# all_df_sincos['day_sin'] = sin_transformer(365).fit_transform(all_df_sincos['day_of_year'])
# all_df_sincos['day_cos'] = cos_transformer(365).fit_transform(all_df_sincos['day_of_year'])


# fig, ax = plt.subplots(3, 1, sharex=True, figsize=(16,8))
# all_df_sincos[["month_sin", "month_cos"]].plot(ax=ax[0])
# all_df_sincos[["weekday_sin", "weekday_cos"]].plot(ax=ax[1])
# all_df_sincos[["day_sin", "day_cos"]].plot(ax=ax[2])
# plt.suptitle("Cyclical encoding with sine/cosine transformation")

In [261]:
# # 放射状基底関数
# all_df_sincos_rbf = all_df_sincos.copy()

# rbf = RepeatingBasisFunction(n_periods=12, column='day_of_year', input_range=(1, 365), remainder='drop')
# rbf.fit(all_df_sincos_rbf)

# tmp_column = [f'month_rbf_{i}' for i in range(1, 13)]
# day_of_year_rbf = pd.DataFrame(columns=tmp_column, data=rbf.transform(all_df_sincos_rbf))

# # 放射状基底関数を結合
# all_df_sincos_rbf = pd.merge(all_df_sincos_rbf, day_of_year_rbf, left_index=True, right_index=True)

# # プロット
# day_of_year_rbf.plot(subplots=True, figsize=(14, 8),
#      	sharex=True, title="Radial Basis Functions",
#      	legend=False)

In [262]:
# # train, test に分ける
# train_df_sincos_rbf = all_df_sincos_rbf.iloc[:350, :]
# test_df_sincos_rbf = all_df_sincos_rbf.iloc[350:, :].dropna(axis=1)

In [263]:
# 検証データのindexを指定(訓練データの最後2ヶ月を使用)
# valid_index = range(297,351) # month: 2,3
valid_index = range(325,351) # month: 3
# valid_index = range(1, 21) # month: 4

In [264]:
# 予測結果を保存する辞書型データ
results = dict({})
all_lgb_score = []

# 商品ごとの予測を作成

for c in tqdm(target_columns):
    # train_tmp = train_df_sincos_rbf.copy()
    train_tmp = train_df.copy()
    # test_tmp = test_df_sincos_rbf.copy()
    test_tmp = test_df.copy()
    
    #ice
    if c in ['ice1', 'ice2', 'ice3']:
        # 予測期間はアイスが人気な7, 8 ,9月ではないので除外
        train_tmp = train_tmp[~train_tmp['month'].isin([7, 8, 9])]
        # アイスは金曜に人気
        train_tmp['is_wday4'] = train_df['weekday'].isin([4]).astype(int)
        test_tmp['is_wday4'] = test_df['weekday'].isin([4]).astype(int)
    # oden
    elif c in ['oden1', 'oden2', 'oden3', 'oden4']:
        # おでんやって無い夏は考えない
        train_tmp = train_tmp[~train_tmp['month'].isin([7, 8, 9])]
        # おでんは水木で人気
        train_tmp['is_wday23'] = train_df['weekday'].isin([2, 3]).astype(int)
        test_tmp['is_wday23'] = test_df['weekday'].isin([2, 3]).astype(int)
    # hot
    elif c in ['hot1', 'hot2', 'hot3']:
        # ホットスナックは月、火、金で不人気
        train_tmp['is_wday014'] = train_df['weekday'].isin([0,1,4]).astype(int)
        test_tmp['is_wday014'] = test_df['weekday'].isin([0,1,4]).astype(int)
    # dessert
    elif c in ['dessert1', 'dessert2', 'dessert3', 'dessert4', 'dessert5']:
        # デザートは水曜と日曜で人気
        train_tmp['is_wday36'] = train_df['weekday'].isin([3,6]).astype(int)
        test_tmp['is_wday36'] = test_df['weekday'].isin([3,6]).astype(int)
    # drink1234
    elif c in ['drink1', 'drink2', 'drink3', 'drink4']:
        # 夏に人気なドリンクは夏のデータは使わない
        train_tmp = train_tmp[~train_tmp['month'].isin([7, 8, 9])]
        if c in ['drink1', 'drink4']:
            # 金曜に人気
            train_tmp['is_wday4'] = train_df['weekday'].isin([4]).astype(int)
            test_tmp['is_wday4'] = test_df['weekday'].isin([4]).astype(int)
        else:
            # 常に人気が変わらないドリンクは全期間つかう。火曜に人気
            train_tmp['is_wday1'] = train_df['weekday'].isin([1]).astype(int)
            test_tmp['is_wday1'] = test_df['weekday'].isin([1]).astype(int)
    # drink5
    elif c in ['drink5']:
        # 金曜に人気
        train_tmp['is_wday4'] = train_df['weekday'].isin([4]).astype(int)
        test_tmp['is_wday4'] = test_df['weekday'].isin([4]).astype(int)
    # drink6
    elif c in ['drink6']:
        # 日曜に人気
        train_tmp['is_wday6'] = train_df['weekday'].isin([6]).astype(int)
        test_tmp['is_wday6'] = test_df['weekday'].isin([6]).astype(int)
    # alcohol
    elif c in ['alcol1', 'alcol2', 'alcol3']:
        # 酒は水木で人気
        train_tmp['is_wday23'] = train_df['weekday'].isin([2, 3]).astype(int)
        test_tmp['is_wday23'] = test_df['weekday'].isin([2, 3]).astype(int)
    # snack
    elif c in ['snack1', 'snack2', 'snack3']:
        train_tmp['is_wday0'] = train_df['weekday'].isin([0]).astype(int)
        train_tmp['is_wday14'] = train_df['weekday'].isin([1,4]).astype(int)
        test_tmp['is_wday0'] = test_df['weekday'].isin([0]).astype(int)
        test_tmp['is_wday14'] = test_df['weekday'].isin([1,4]).astype(int)
    # bento
    elif c in ['bento1', 'bento2', 'bento3', 'bento4']:
        # 弁当は月、火、金で不人気
        train_tmp['is_wday014'] = train_df['weekday'].isin([0,1,4]).astype(int)
        test_tmp['is_wday014'] = test_df['weekday'].isin([0,1,4]).astype(int)
    # tild
    elif c in ['tild1', 'tild2']:
        # 日曜に人気
        train_tmp['is_wday6'] = train_df['weekday'].isin([6]).astype(int)
        test_tmp['is_wday6'] = test_df['weekday'].isin([6]).astype(int)
    # men
    elif c in ['men1', 'men2', 'men3', 'men4', 'men5', 'men6']:
        train_tmp['is_wday014'] = train_df['weekday'].isin([0,1,4]).astype(int)
        test_tmp['is_wday014'] = test_df['weekday'].isin([0,1,4]).astype(int)
    
    # 訓練に確実に不必要なデータを除く
    train_columns = [c for c in train_tmp.columns if c not in target_columns if c not in\
    # ['id', 'date', 'time', 'month', 'year', 'day', 'weekday', 'day_of_year']]
        ['id', 'date', 'time']]
    
    # データを訓練、検証に分割
    X_train = train_tmp[~train_tmp['id'].isin(valid_index)][train_columns]
    y_train = train_tmp[~train_tmp['id'].isin(valid_index)][c]
    X_valid = train_tmp[train_tmp['id'].isin(valid_index)][train_columns]
    y_valid = train_tmp[train_tmp['id'].isin(valid_index)][c]
     
    # テストデータ
    X_test = test_tmp[train_columns]

    # 分位点
    qs = np.array([0.01, 0.1, 0.5, 0.9, 0.99])
    
    # 分位点ごとのスコア
    q_scores = []
    
    # 分位点ごとにモデルを作成
    for q in qs:
        lgb = LGBMRegressor(
            objective='quantile',
            alpha = q,
            n_estimators=10000,
            colsample_bytree=0.9,
            random_state=seed,
            max_depth=5,
        )
        
        # # RFEにより特徴量選択
        # selector = RFE(lgb, n_features_to_select=0.6)
        # selector.fit(X_train, y_train)
        
        # # 特徴量が生き残ったか（残った：True、ドロップ：False）を表示
        # result = pd.DataFrame(selector.get_support(), index=X_train.columns.values, columns=['left'])
        # result['ranking'] = selector.ranking_
        # print(f'---{c}_{q}---')
        # display(result.T)
        # print(' ')
        
        # このモデルで採用されたカラム
        # train_columns_RFE = X_train.columns.values[selector.get_support()]
        
        # # 採用されたカラムだけでデータを作成
        # X_train_RFE = X_train[train_columns_RFE]
        # X_valid_RFE = X_valid[train_columns_RFE]
        # X_test_RFE = X_test[train_columns_RFE]
        
        
        # 学習
        lgb.fit(X_train, y_train, eval_set=(X_valid, y_valid), early_stopping_rounds=100, verbose=False)
        score = lgb.best_score_['valid_0']['quantile']
        
        y_pred = lgb.predict(X_test)
        results[(c, q)] = y_pred
        
        q_scores.append(score)

    all_lgb_score.append(q_scores)

score_df_1 = pd.DataFrame(np.array(all_lgb_score), columns=qs, index=target_columns)
display(score_df_1)

print(f'ave score: {np.array(all_lgb_score).mean()}')

  0%|          | 0/39 [00:00<?, ?it/s]

,0.01,0.10,0.50,0.90,0.99
ice1,0.035231,0.261459,0.808972,0.575586,0.089615
ice2,0.171643,0.343730,0.433284,0.879530,0.228448
ice3,0.054523,0.448462,1.378846,0.444239,0.080227
oden1,0.473534,1.086877,2.844364,1.548829,0.278879
oden2,0.248911,1.046709,3.141214,1.528414,0.281752
oden3,0.381475,1.749487,4.285037,2.609950,0.661377
oden4,0.348506,1.569710,4.673192,4.071986,1.083020
hot1,0.271771,1.226096,3.660201,1.334646,0.745947
hot2,0.256462,2.148908,6.363941,2.729525,0.351232
hot3,0.312402,2.384146,5.522259,2.271541,0.210540


ave score: 0.7412706596597354


In [265]:
submit_rows = [[f'{k[0]}_{k[1]}'] + v.tolist() for k, v in results.items()]

In [266]:
# submitファイルを作成
# データを整数に丸めることでわずかに精度が上がるらしい
submit_df_1 = pd.DataFrame(np.array(submit_rows)[:, 1:22].astype(float), index=np.array(submit_rows)[:, 0], columns=range(1, 22))
submit_df_1_round = pd.DataFrame(np.array(submit_rows)[:, 1:22].astype(float).round(), index=np.array(submit_rows)[:, 0], columns=range(1, 22))

# 転置
submit_df_1_round = submit_df_1_round.transpose()
submit_df_1 = submit_df_1.transpose()

submit_df_1.index.name = 'id'
submit_df_1_round.index.name = 'id'

# submit_df_1_round.to_csv("../work/submission/sub_lgb_kotrying_plus_day_of_year_valid3_max_depth5_true_index.csv")
display(submit_df_1_round)

,ice1_0.01,ice1_0.1,ice1_0.5,ice1_0.9,ice1_0.99,ice2_0.01,ice2_0.1,ice2_0.5,ice2_0.9,ice2_0.99,ice3_0.01,ice3_0.1,ice3_0.5,ice3_0.9,ice3_0.99,oden1_0.01,oden1_0.1,oden1_0.5,oden1_0.9,oden1_0.99,oden2_0.01,oden2_0.1,oden2_0.5,oden2_0.9,oden2_0.99,oden3_0.01,oden3_0.1,oden3_0.5,oden3_0.9,oden3_0.99,oden4_0.01,oden4_0.1,oden4_0.5,oden4_0.9,oden4_0.99,hot1_0.01,hot1_0.1,hot1_0.5,hot1_0.9,hot1_0.99,hot2_0.01,hot2_0.1,hot2_0.5,hot2_0.9,hot2_0.99,hot3_0.01,hot3_0.1,hot3_0.5,hot3_0.9,hot3_0.99,dessert1_0.01,dessert1_0.1,dessert1_0.5,dessert1_0.9,dessert1_0.99,dessert2_0.01,dessert2_0.1,dessert2_0.5,dessert2_0.9,dessert2_0.99,dessert3_0.01,dessert3_0.1,dessert3_0.5,dessert3_0.9,dessert3_0.99,dessert4_0.01,dessert4_0.1,dessert4_0.5,dessert4_0.9,dessert4_0.99,dessert5_0.01,dessert5_0.1,dessert5_0.5,dessert5_0.9,dessert5_0.99,drink1_0.01,drink1_0.1,drink1_0.5,drink1_0.9,drink1_0.99,drink2_0.01,drink2_0.1,drink2_0.5,drink2_0.9,drink2_0.99,drink3_0.01,drink3_0.1,drink3_0.5,drink3_0.9,drink3_0.99,drink4_0.01,drink4_0.1,drink4_0.5,drink4_0.9,drink4_0.99,drink5_0.01,drink5_0.1,drink5_0.5,drink5_0.9,drink5_0.99,drink6_0.01,drink6_0.1,drink6_0.5,drink6_0.9,drink6_0.99,alcol1_0.01,alcol1_0.1,alcol1_0.5,alcol1_0.9,alcol1_0.99,alcol2_0.01,alcol2_0.1,alcol2_0.5,alcol2_0.9,alcol2_0.99,alcol3_0.01,alcol3_0.1,alcol3_0.5,alcol3_0.9,alcol3_0.99,snack1_0.01,snack1_0.1,snack1_0.5,snack1_0.9,snack1_0.99,snack2_0.01,snack2_0.1,snack2_0.5,snack2_0.9,snack2_0.99,snack3_0.01,snack3_0.1,snack3_0.5,snack3_0.9,snack3_0.99,bento1_0.01,bento1_0.1,bento1_0.5,bento1_0.9,bento1_0.99,bento2_0.01,bento2_0.1,bento2_0.5,bento2_0.9,bento2_0.99,bento3_0.01,bento3_0.1,bento3_0.5,bento3_0.9,bento3_0.99,bento4_0.01,bento4_0.1,bento4_0.5,bento4_0.9,bento4_0.99,tild1_0.01,tild1_0.1,tild1_0.5,tild1_0.9,tild1_0.99,tild2_0.01,tild2_0.1,tild2_0.5,tild2_0.9,tild2_0.99,men1_0.01,men1_0.1,men1_0.5,men1_0.9,men1_0.99,men2_0.01,men2_0.1,men2_0.5,men2_0.9,men2_0.99,men3_0.01,men3_0.1,men3_0.5,men3_0.9,men3_0.99,men4_0.01,men4_0.1,men4_0.5,men4_0.9,men4_0.99,men5_0.01,men5_0.1,men5_0.5,men5_0.9,men5_0.99,men6_0.01,men6_0.1,men6_0.5,men6_0.9,men6_0.99
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,16.0,19.0,21.0,22.0,29.0,34.0,61.0,64.0,64.0,76.0,24.0,26.0,31.0,34.0,37.0,6.0,34.0,39.0,45.0,80.0,21.0,40.0,52.0,64.0,91.0,56.0,97.0,106.0,116.0,182.0,17.0,47.0,63.0,68.0,144.0,116.0,136.0,157.0,176.0,211.0,203.0,220.0,228.0,240.0,259.0,194.0,223.0,248.0,252.0,275.0,19.0,47.0,50.0,54.0,56.0,36.0,65.0,74.0,81.0,81.0,7.0,50.0,54.0,58.0,57.0,48.0,54.0,61.0,64.0,67.0,42.0,59.0,77.0,83.0,87.0,19.0,36.0,39.0,38.0,41.0,19.0,31.0,35.0,38.0,54.0,11.0,20.0,23.0,22.0,43.0,27.0,39.0,42.0,45.0,48.0,22.0,23.0,25.0,26.0,26.0,7.0,8.0,8.0,9.0,16.0,49.0,55.0,60.0,61.0,85.0,42.0,44.0,47.0,51.0,128.0,53.0,53.0,56.0,60.0,114.0,18.0,23.0,28.0,30.0,31.0,10.0,17.0,20.0,23.0,31.0,27.0,35.0,41.0,45.0,58.0,42.0,45.0,51.0,53.0,57.0,34.0,37.0,44.0,48.0,50.0,53.0,63.0,72.0,79.0,78.0,9.0,25.0,28.0,30.0,33.0,4.0,11.0,12.0,21.0,20.0,9.0,9.0,10.0,15.0,15.0,42.0,43.0,52.0,58.0,63.0,26.0,27.0,30.0,36.0,34.0,28.0,30.0,37.0,46.0,50.0,26.0,35.0,39.0,50.0,59.0,30.0,33.0,37.0,46.0,49.0,28.0,33.0,38.0,45.0,61.0
2,16.0,18.0,20.0,21.0,29.0,34.0,55.0,57.0,58.0,75.0,24.0,26.0,31.0,32.0,37.0,9.0,23.0,25.0,37.0,80.0,31.0,36.0,27.0,64.0,91.0,72.0,79.0,76.0,89.0,182.0,32.0,49.0,43.0,57.0,144.0,110.0,135.0,149.0,153.0,206.0,200.0,209.0,228.0,233.0,245.0,189.0,204.0,229.0,240.0,264.0,19.0,28.0,32.0,32.0,32.0,36.0,49.0,53.0,57.0,62.0,5.0,9.0,10.0,11.0,19.0,27.0,29.0,32.0,36.0,49.0,33.0,35.0,38.0,48.0,42.0,19.0,25.0,27.0,36.0,42.0,19.0,26.0,27.0,31.0,53.0,11.0,17.0,20.0,22.0,28.0,27.0,33.0,35.0,39.0,48.0,22.0,23.0,25.0,26.0,26.0,7.0,7.0,8.0,9.0,16.0,67.0,66.0,70.0,74.0,87.0,57.0,56.0,63.0,68.0,128.0,63.0,62.0,73.0,79.0,114.0,20.0,29.0,33.0,38.0,39.0,9.0,28.0,33.0,36.0,37.0,27.0,42.0,55.0,61.0,67.0,8.

In [267]:
results = dict({})
all_lgb_score_2 = []

for c in tqdm(target_columns):
    
    # ある商品の全スコア 
    q_scores = []
    
    # 21日個別のモデルを作る
    for i in range(1, 22):
        
        all_tmp = all_df.copy()
    
        # おでんの場合は7, 8, 9月を考えない
        if c in ['oden1', 'oden2', 'oden3', 'oden4']:
            # おでんやって無い夏は考えない
            all_tmp = all_tmp[~all_tmp['month'].isin([7, 8, 9])]
        
        if i <= 7:
            # 10週のラグ特徴量を考える
            for j in range(1, 11):
                all_tmp[f'lag_{j}week'] = np.log1p(all_tmp[c].shift(7*j))
                # ラグ特徴量の5週間の平均
            all_tmp[f'lag_ave_5week'] = all_tmp[[f'lag_{k}week'for k in range(1, 6)]].mean(axis=1)
            
        elif i > 7 and i <= 14:
            for j in range(1, 11):
                all_tmp[f'lag_{j+1}week'] = np.log1p(all_tmp[c].shift(7*(j+1)))
            all_tmp[f'lag_ave_5week'] = all_tmp[[f'lag_{k}week'for k in range(2, 7)]].mean(axis=1)
        elif i > 14:
            for j in range(1, 11):
                all_tmp[f'lag_{j+2}week'] = np.log1p(all_tmp[c].shift(7*(j+2)))
            all_tmp[f'lag_ave_5week'] = all_tmp[[f'lag_{k}week'for k in range(3, 8)]].mean(axis=1)
        
        # 最新20日のデータ
        cnt = 1
        while cnt <= 20:
            if (cnt+i-1) % 7 == 0:
                cnt += 1
                continue
            all_tmp[f'last_{cnt}day'] = np.log1p(all_tmp[c].shift(cnt+i-1))
            cnt += 1
            
        # データ分割 + 訓練データの最初のNaNをけす
        train_tmp = all_tmp[all_tmp.index<350]
        train_tmp = train_tmp.dropna(axis=0)
        
        test_tmp = all_tmp[all_tmp.index==350+i-1]
        test_tmp = test_tmp.dropna(axis=1)
        
        # 除く特徴量
        removed_cols = ['date', 'id', 'time']
        train_columns = [col for col in train_tmp.columns if col not in target_columns if col not in removed_cols]
        X_train = train_tmp[~train_tmp['id'].isin(valid_index)][train_columns]
        y_train = train_tmp[~train_tmp['id'].isin(valid_index)][c]
        X_valid = train_tmp[train_tmp['id'].isin(valid_index)][train_columns]
        y_valid = train_tmp[train_tmp['id'].isin(valid_index)][c]
        
        X_test = test_tmp[train_columns]
    
        
        qs = np.array([0.01, 0.1, 0.5, 0.9, 0.99])
    
        # 分位点ごとのスコア
        i_q_scores = []
        
       
        # 分位点ごとにモデルを作成
        for q in qs:
            lgb = LGBMRegressor(
                objective='quantile',
                alpha = q,
                n_estimators=10000,
                colsample_bytree=0.9,
                random_state=seed,
                max_depth=5,
            )
            
            lgb.fit(X_train, y_train, eval_set=(X_valid, y_valid), early_stopping_rounds=100, verbose=False)
            
            # 商品c, i日目, 分位点q のスコア
            score = lgb.best_score_['valid_0']['quantile']
        
            y_pred = lgb.predict(X_test)
            results[(i, c, q)] = y_pred

            # 商品c, i日目のスコア配列
            i_q_scores.append(score)

        q_scores.append(i_q_scores)
        
    q_scores = np.array(q_scores)
        
    # ある商品のqごとのスコア(21日を平均)
    q_score = np.mean(q_scores, axis=0)
        
    all_lgb_score_2.append(list(q_score))

score_df_2 = pd.DataFrame(np.array(all_lgb_score_2), columns=qs, index=target_columns)
display(score_df_2)

print(f'ave score: {np.array(all_lgb_score_2).mean()}')
                  

  0%|          | 0/39 [00:00<?, ?it/s]

,0.01,0.10,0.50,0.90,0.99
ice1,0.034589,0.293310,0.887779,0.612332,0.203798
ice2,0.150744,0.692832,0.610982,0.655482,0.380730
ice3,0.054469,0.451848,1.226696,0.486556,0.131736
oden1,0.423899,2.381323,3.134142,1.681846,0.409222
oden2,0.370355,1.899024,3.308608,3.003890,0.758487
oden3,0.589213,3.989400,6.221040,4.845928,0.773575
oden4,0.472182,2.439459,4.766878,3.800999,0.981763
hot1,0.300875,1.946313,4.647421,3.026752,0.792817
hot2,0.361233,2.873714,7.339550,3.344670,0.425202
hot3,0.335775,2.768911,6.440214,3.224062,0.425097


ave score: 0.965330592227585


In [268]:
# 提出ファイル作成
submit_df_2 = submission_df.copy()
submit_df_2_round = submission_df.copy()
for i in range(21):
    for c in target_columns:
        for q in qs:
            submit_df_2.loc[i, f'{c}_{q}'] = results[(i+1, c, q)]
            submit_df_2_round.loc[i, f'{c}_{q}'] = results[(i+1, c, q)].round()

submit_df_2_round = submit_df_2_round.set_index('id')
submit_df_2 = submit_df_2.set_index('id')

In [269]:
submit_df_2_round.to_csv("../work/submission/sub_lgb_lag_first_valid3.csv")
display(submit_df_2_round)

,ice1_0.01,ice1_0.1,ice1_0.5,ice1_0.9,ice1_0.99,ice2_0.01,ice2_0.1,ice2_0.5,ice2_0.9,ice2_0.99,ice3_0.01,ice3_0.1,ice3_0.5,ice3_0.9,ice3_0.99,oden1_0.01,oden1_0.1,oden1_0.5,oden1_0.9,oden1_0.99,oden2_0.01,oden2_0.1,oden2_0.5,oden2_0.9,oden2_0.99,oden3_0.01,oden3_0.1,oden3_0.5,oden3_0.9,oden3_0.99,oden4_0.01,oden4_0.1,oden4_0.5,oden4_0.9,oden4_0.99,hot1_0.01,hot1_0.1,hot1_0.5,hot1_0.9,hot1_0.99,hot2_0.01,hot2_0.1,hot2_0.5,hot2_0.9,hot2_0.99,hot3_0.01,hot3_0.1,hot3_0.5,hot3_0.9,hot3_0.99,dessert1_0.01,dessert1_0.1,dessert1_0.5,dessert1_0.9,dessert1_0.99,dessert2_0.01,dessert2_0.1,dessert2_0.5,dessert2_0.9,dessert2_0.99,dessert3_0.01,dessert3_0.1,dessert3_0.5,dessert3_0.9,dessert3_0.99,dessert4_0.01,dessert4_0.1,dessert4_0.5,dessert4_0.9,dessert4_0.99,dessert5_0.01,dessert5_0.1,dessert5_0.5,dessert5_0.9,dessert5_0.99,drink1_0.01,drink1_0.1,drink1_0.5,drink1_0.9,drink1_0.99,drink2_0.01,drink2_0.1,drink2_0.5,drink2_0.9,drink2_0.99,drink3_0.01,drink3_0.1,drink3_0.5,drink3_0.9,drink3_0.99,drink4_0.01,drink4_0.1,drink4_0.5,drink4_0.9,drink4_0.99,drink5_0.01,drink5_0.1,drink5_0.5,drink5_0.9,drink5_0.99,drink6_0.01,drink6_0.1,drink6_0.5,drink6_0.9,drink6_0.99,alcol1_0.01,alcol1_0.1,alcol1_0.5,alcol1_0.9,alcol1_0.99,alcol2_0.01,alcol2_0.1,alcol2_0.5,alcol2_0.9,alcol2_0.99,alcol3_0.01,alcol3_0.1,alcol3_0.5,alcol3_0.9,alcol3_0.99,snack1_0.01,snack1_0.1,snack1_0.5,snack1_0.9,snack1_0.99,snack2_0.01,snack2_0.1,snack2_0.5,snack2_0.9,snack2_0.99,snack3_0.01,snack3_0.1,snack3_0.5,snack3_0.9,snack3_0.99,bento1_0.01,bento1_0.1,bento1_0.5,bento1_0.9,bento1_0.99,bento2_0.01,bento2_0.1,bento2_0.5,bento2_0.9,bento2_0.99,bento3_0.01,bento3_0.1,bento3_0.5,bento3_0.9,bento3_0.99,bento4_0.01,bento4_0.1,bento4_0.5,bento4_0.9,bento4_0.99,tild1_0.01,tild1_0.1,tild1_0.5,tild1_0.9,tild1_0.99,tild2_0.01,tild2_0.1,tild2_0.5,tild2_0.9,tild2_0.99,men1_0.01,men1_0.1,men1_0.5,men1_0.9,men1_0.99,men2_0.01,men2_0.1,men2_0.5,men2_0.9,men2_0.99,men3_0.01,men3_0.1,men3_0.5,men3_0.9,men3_0.99,men4_0.01,men4_0.1,men4_0.5,men4_0.9,men4_0.99,men5_0.01,men5_0.1,men5_0.5,men5_0.9,men5_0.99,men6_0.01,men6_0.1,men6_0.5,men6_0.9,men6_0.99
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,17.0,20.0,21.0,22.0,74.0,33.0,60.0,63.0,64.0,101.0,24.0,26.0,29.0,34.0,44.0,6.0,20.0,41.0,56.0,107.0,20.0,35.0,48.0,64.0,94.0,50.0,76.0,86.0,160.0,194.0,31.0,46.0,52.0,73.0,147.0,102.0,133.0,149.0,185.0,205.0,178.0,200.0,219.0,255.0,266.0,187.0,208.0,222.0,258.0,266.0,19.0,38.0,53.0,53.0,54.0,36.0,55.0,74.0,71.0,88.0,5.0,43.0,52.0,62.0,59.0,31.0,58.0,63.0,63.0,67.0,32.0,60.0,62.0,75.0,76.0,13.0,35.0,39.0,40.0,69.0,17.0,28.0,34.0,40.0,56.0,9.0,20.0,21.0,23.0,39.0,27.0,36.0,42.0,45.0,52.0,22.0,23.0,24.0,26.0,29.0,7.0,8.0,8.0,9.0,16.0,55.0,57.0,63.0,62.0,93.0,40.0,45.0,55.0,53.0,127.0,37.0,54.0,56.0,62.0,108.0,18.0,22.0,25.0,28.0,36.0,9.0,17.0,17.0,20.0,26.0,27.0,36.0,45.0,47.0,58.0,8.0,46.0,50.0,53.0,61.0,12.0,36.0,44.0,45.0,58.0,17.0,63.0,71.0,77.0,80.0,10.0,27.0,29.0,31.0,33.0,4.0,11.0,11.0,15.0,20.0,7.0,9.0,11.0,12.0,18.0,17.0,46.0,52.0,62.0,64.0,17.0,26.0,32.0,31.0,36.0,14.0,34.0,35.0,41.0,54.0,26.0,35.0,38.0,46.0,49.0,4.0,36.0,40.0,45.0,50.0,15.0,34.0,38.0,47.0,72.0
2,15.0,18.0,19.0,21.0,52.0,34.0,51.0,55.0,55.0,98.0,24.0,26.0,29.0,34.0,39.0,6.0,14.0,24.0,60.0,92.0,23.0,34.0,36.0,65.0,106.0,53.0,76.0,78.0,162.0,199.0,16.0,52.0,53.0,64.0,147.0,101.0,131.0,146.0,180.0,220.0,178.0,196.0,227.0,247.0,259.0,185.0,202.0,225.0,240.0,272.0,19.0,22.0,29.0,37.0,40.0,36.0,41.0,53.0,58.0,80.0,5.0,5.0,12.0,17.0,29.0,25.0,27.0,31.0,37.0,65.0,32.0,33.0,38.0,52.0,72.0,14.0,23.0,25.0,29.0,88.0,17.0,26.0,28.0,39.0,52.0,9.0,14.0,19.0,27.0,46.0,27.0,31.0,33.0,34.0,49.0,22.0,22.0,25.0,26.0,27.0,7.0,7.0,8.0,9.0,16.0,61.0,70.0,74.0,79.0,95.0,36.0,59.0,68.0,74.0,127.0,37.0,66.0,72.0,80.0,108.0,18.0,26.0,31.0,39.0,39.0,9.0,28.0,32.0,34.0,36.0,27.0,39.0,52.0,57.0,69.0,8.0

In [270]:
submit_df_1.head()

,ice1_0.01,ice1_0.1,ice1_0.5,ice1_0.9,ice1_0.99,ice2_0.01,ice2_0.1,ice2_0.5,ice2_0.9,ice2_0.99,ice3_0.01,ice3_0.1,ice3_0.5,ice3_0.9,ice3_0.99,oden1_0.01,oden1_0.1,oden1_0.5,oden1_0.9,oden1_0.99,oden2_0.01,oden2_0.1,oden2_0.5,oden2_0.9,oden2_0.99,oden3_0.01,oden3_0.1,oden3_0.5,oden3_0.9,oden3_0.99,oden4_0.01,oden4_0.1,oden4_0.5,oden4_0.9,oden4_0.99,hot1_0.01,hot1_0.1,hot1_0.5,hot1_0.9,hot1_0.99,hot2_0.01,hot2_0.1,hot2_0.5,hot2_0.9,hot2_0.99,hot3_0.01,hot3_0.1,hot3_0.5,hot3_0.9,hot3_0.99,dessert1_0.01,dessert1_0.1,dessert1_0.5,dessert1_0.9,dessert1_0.99,dessert2_0.01,dessert2_0.1,dessert2_0.5,dessert2_0.9,dessert2_0.99,dessert3_0.01,dessert3_0.1,dessert3_0.5,dessert3_0.9,dessert3_0.99,dessert4_0.01,dessert4_0.1,dessert4_0.5,dessert4_0.9,dessert4_0.99,dessert5_0.01,dessert5_0.1,dessert5_0.5,dessert5_0.9,dessert5_0.99,drink1_0.01,drink1_0.1,drink1_0.5,drink1_0.9,drink1_0.99,drink2_0.01,drink2_0.1,drink2_0.5,drink2_0.9,drink2_0.99,drink3_0.01,drink3_0.1,drink3_0.5,drink3_0.9,drink3_0.99,drink4_0.01,drink4_0.1,drink4_0.5,drink4_0.9,drink4_0.99,drink5_0.01,drink5_0.1,drink5_0.5,drink5_0.9,drink5_0.99,drink6_0.01,drink6_0.1,drink6_0.5,drink6_0.9,drink6_0.99,alcol1_0.01,alcol1_0.1,alcol1_0.5,alcol1_0.9,alcol1_0.99,alcol2_0.01,alcol2_0.1,alcol2_0.5,alcol2_0.9,alcol2_0.99,alcol3_0.01,alcol3_0.1,alcol3_0.5,alcol3_0.9,alcol3_0.99,snack1_0.01,snack1_0.1,snack1_0.5,snack1_0.9,snack1_0.99,snack2_0.01,snack2_0.1,snack2_0.5,snack2_0.9,snack2_0.99,snack3_0.01,snack3_0.1,snack3_0.5,snack3_0.9,snack3_0.99,bento1_0.01,bento1_0.1,bento1_0.5,bento1_0.9,bento1_0.99,bento2_0.01,bento2_0.1,bento2_0.5,bento2_0.9,bento2_0.99,bento3_0.01,bento3_0.1,bento3_0.5,bento3_0.9,bento3_0.99,bento4_0.01,bento4_0.1,bento4_0.5,bento4_0.9,bento4_0.99,tild1_0.01,tild1_0.1,tild1_0.5,tild1_0.9,tild1_0.99,tild2_0.01,tild2_0.1,tild2_0.5,tild2_0.9,tild2_0.99,men1_0.01,men1_0.1,men1_0.5,men1_0.9,men1_0.99,men2_0.01,men2_0.1,men2_0.5,men2_0.9,men2_0.99,men3_0.01,men3_0.1,men3_0.5,men3_0.9,men3_0.99,men4_0.01,men4_0.1,men4_0.5,men4_0.9,men4_0.99,men5_0.01,men5_0.1,men5_0.5,men5_0.9,men5_0.99,men6_0.01,men6_0.1,men6_0.5,men6_0.9,men6_0.99
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,15.958576,19.167259,21.009127,21.801729,29.048276,34.000000,61.186192,63.566230,64.055123,76.150242,24.10,26.1,31.0,33.715031,37.20307,6.000000,34.409704,39.220382,45.048735,79.994045,21.180968,39.893868,52.457657,63.726166,91.078856,55.962991,96.956021,105.573085,116.316885,181.864365,17.206436,47.273085,63.410879,67.683243,144.002007,115.866398,136.343354,157.144877,175.702727,211.042876,202.740225,220.410433,228.157046,240.168730,258.838461,193.540201,222.938696,248.045654,251.541784,275.384355,19.2,47.145874,50.007084,54.091328,56.148918,36.1,64.546838,74.291725,80.828138,81.026226,6.927938,50.287043,54.281123,57.504475,57.460877,48.019219,54.360070,61.158750,63.701231,66.770150,41.521341,59.040216,77.252396,82.973561,86.725562,19.042621,36.156585,38.761888,38.069910,41.422549,19.406899,31.338245,35.209891,37.621626,54.085882,10.670347,20.407379,22.946312,22.079277,43.195397,27.22,38.858041,42.119112,44.785523,48.107334,22.1,22.891905,24.71,26.077247,26.330008,7.159115,7.652655,8.009845,9.490591,16.000000,49.363728,54.625296,59.918804,61.282698,84.970564,41.652493,43.835142,46.705897,50.541482,127.620216,52.908910,52.886684,56.032279,59.866304,113.555698,18.326724,22.630431,27.763757,30.434838,30.757750,9.843812,16.598226,19.547227,23.263131,31.088481,27.291098,34.530000,40.602594,45.456881,58.000000,41.918724,44.970728,51.312755,53.432322,56.944295,34.277298,36.840249,43.683647,47.606572,49.635402,53.495922,62.667049,72.280888,78.572789,77.615533,9.09888,25.163488,27.858591,30.117937,33.062230,4.10,11.266002,12.305946,21.066655,19.861788,9.223481,9.147497,10.342350,14.638324,15.0,42.476194,42.618847,52.223270,57.689595,63.302300,25.576127,26.7

In [271]:
submit_df_2.head()

,ice1_0.01,ice1_0.1,ice1_0.5,ice1_0.9,ice1_0.99,ice2_0.01,ice2_0.1,ice2_0.5,ice2_0.9,ice2_0.99,ice3_0.01,ice3_0.1,ice3_0.5,ice3_0.9,ice3_0.99,oden1_0.01,oden1_0.1,oden1_0.5,oden1_0.9,oden1_0.99,oden2_0.01,oden2_0.1,oden2_0.5,oden2_0.9,oden2_0.99,oden3_0.01,oden3_0.1,oden3_0.5,oden3_0.9,oden3_0.99,oden4_0.01,oden4_0.1,oden4_0.5,oden4_0.9,oden4_0.99,hot1_0.01,hot1_0.1,hot1_0.5,hot1_0.9,hot1_0.99,hot2_0.01,hot2_0.1,hot2_0.5,hot2_0.9,hot2_0.99,hot3_0.01,hot3_0.1,hot3_0.5,hot3_0.9,hot3_0.99,dessert1_0.01,dessert1_0.1,dessert1_0.5,dessert1_0.9,dessert1_0.99,dessert2_0.01,dessert2_0.1,dessert2_0.5,dessert2_0.9,dessert2_0.99,dessert3_0.01,dessert3_0.1,dessert3_0.5,dessert3_0.9,dessert3_0.99,dessert4_0.01,dessert4_0.1,dessert4_0.5,dessert4_0.9,dessert4_0.99,dessert5_0.01,dessert5_0.1,dessert5_0.5,dessert5_0.9,dessert5_0.99,drink1_0.01,drink1_0.1,drink1_0.5,drink1_0.9,drink1_0.99,drink2_0.01,drink2_0.1,drink2_0.5,drink2_0.9,drink2_0.99,drink3_0.01,drink3_0.1,drink3_0.5,drink3_0.9,drink3_0.99,drink4_0.01,drink4_0.1,drink4_0.5,drink4_0.9,drink4_0.99,drink5_0.01,drink5_0.1,drink5_0.5,drink5_0.9,drink5_0.99,drink6_0.01,drink6_0.1,drink6_0.5,drink6_0.9,drink6_0.99,alcol1_0.01,alcol1_0.1,alcol1_0.5,alcol1_0.9,alcol1_0.99,alcol2_0.01,alcol2_0.1,alcol2_0.5,alcol2_0.9,alcol2_0.99,alcol3_0.01,alcol3_0.1,alcol3_0.5,alcol3_0.9,alcol3_0.99,snack1_0.01,snack1_0.1,snack1_0.5,snack1_0.9,snack1_0.99,snack2_0.01,snack2_0.1,snack2_0.5,snack2_0.9,snack2_0.99,snack3_0.01,snack3_0.1,snack3_0.5,snack3_0.9,snack3_0.99,bento1_0.01,bento1_0.1,bento1_0.5,bento1_0.9,bento1_0.99,bento2_0.01,bento2_0.1,bento2_0.5,bento2_0.9,bento2_0.99,bento3_0.01,bento3_0.1,bento3_0.5,bento3_0.9,bento3_0.99,bento4_0.01,bento4_0.1,bento4_0.5,bento4_0.9,bento4_0.99,tild1_0.01,tild1_0.1,tild1_0.5,tild1_0.9,tild1_0.99,tild2_0.01,tild2_0.1,tild2_0.5,tild2_0.9,tild2_0.99,men1_0.01,men1_0.1,men1_0.5,men1_0.9,men1_0.99,men2_0.01,men2_0.1,men2_0.5,men2_0.9,men2_0.99,men3_0.01,men3_0.1,men3_0.5,men3_0.9,men3_0.99,men4_0.01,men4_0.1,men4_0.5,men4_0.9,men4_0.99,men5_0.01,men5_0.1,men5_0.5,men5_0.9,men5_0.99,men6_0.01,men6_0.1,men6_0.5,men6_0.9,men6_0.99
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,16.918091,20.074568,20.989497,21.708897,74.159182,32.708765,59.646185,63.161276,63.753090,100.848894,24.10,26.260001,28.906944,33.719987,43.583899,6.375413,20.206320,41.086053,56.458747,106.909167,19.530011,35.152228,48.079502,63.697659,93.754444,50.282332,76.220214,86.378500,160.222824,194.244802,30.849341,45.540556,52.397690,73.312369,147.326004,101.538472,133.278363,149.372644,184.902673,205.081852,178.333813,200.491762,219.315907,255.334809,266.278122,186.955064,207.553681,222.058295,257.623320,265.643619,19.11,38.457439,52.797230,52.804016,53.777111,36.147,54.862107,73.597829,71.233186,87.915365,5.100000,42.923639,52.034947,62.075883,58.586076,31.340667,57.905976,62.625325,62.976697,67.033527,32.1,59.565673,62.135564,75.319764,76.161986,13.319412,35.242493,39.115205,40.107118,69.397843,16.993844,28.478304,33.868873,39.834980,55.876281,9.25,19.535854,20.976838,22.554006,39.244490,27.1,35.719817,42.393796,44.640405,52.475436,22.0,22.971225,24.286271,26.143067,28.606344,7.177075,7.840408,8.114212,8.895396,16.000000,55.413679,57.414978,62.638529,62.284556,93.294455,39.618915,45.036656,54.567124,52.731192,127.024011,37.125979,53.894146,55.992001,61.804667,108.115,18.462222,22.373062,24.638433,28.141985,35.927094,9.274104,16.776198,16.866400,19.569415,26.449991,26.635994,36.371604,44.613260,47.053910,58.000000,8.1,46.120091,50.398329,52.990283,60.551218,11.639564,36.458697,44.097675,44.717344,57.723475,16.645856,63.365673,70.860189,77.218438,80.048187,10.328449,26.815858,28.540013,30.615054,33.000000,4.100000,10.589642,10.907895,15.34000,19.724765,6.983632,8.842548,11.022369,11.642493,18.0,17.100,46.268798,52.040353,61.517153,64.148931,16.780962,26.

In [272]:
# アンサンブル
df_ans = (submit_df_1 + submit_df_2) / 2
df_ans = df_ans.round()
df_ans

,ice1_0.01,ice1_0.1,ice1_0.5,ice1_0.9,ice1_0.99,ice2_0.01,ice2_0.1,ice2_0.5,ice2_0.9,ice2_0.99,ice3_0.01,ice3_0.1,ice3_0.5,ice3_0.9,ice3_0.99,oden1_0.01,oden1_0.1,oden1_0.5,oden1_0.9,oden1_0.99,oden2_0.01,oden2_0.1,oden2_0.5,oden2_0.9,oden2_0.99,oden3_0.01,oden3_0.1,oden3_0.5,oden3_0.9,oden3_0.99,oden4_0.01,oden4_0.1,oden4_0.5,oden4_0.9,oden4_0.99,hot1_0.01,hot1_0.1,hot1_0.5,hot1_0.9,hot1_0.99,hot2_0.01,hot2_0.1,hot2_0.5,hot2_0.9,hot2_0.99,hot3_0.01,hot3_0.1,hot3_0.5,hot3_0.9,hot3_0.99,dessert1_0.01,dessert1_0.1,dessert1_0.5,dessert1_0.9,dessert1_0.99,dessert2_0.01,dessert2_0.1,dessert2_0.5,dessert2_0.9,dessert2_0.99,dessert3_0.01,dessert3_0.1,dessert3_0.5,dessert3_0.9,dessert3_0.99,dessert4_0.01,dessert4_0.1,dessert4_0.5,dessert4_0.9,dessert4_0.99,dessert5_0.01,dessert5_0.1,dessert5_0.5,dessert5_0.9,dessert5_0.99,drink1_0.01,drink1_0.1,drink1_0.5,drink1_0.9,drink1_0.99,drink2_0.01,drink2_0.1,drink2_0.5,drink2_0.9,drink2_0.99,drink3_0.01,drink3_0.1,drink3_0.5,drink3_0.9,drink3_0.99,drink4_0.01,drink4_0.1,drink4_0.5,drink4_0.9,drink4_0.99,drink5_0.01,drink5_0.1,drink5_0.5,drink5_0.9,drink5_0.99,drink6_0.01,drink6_0.1,drink6_0.5,drink6_0.9,drink6_0.99,alcol1_0.01,alcol1_0.1,alcol1_0.5,alcol1_0.9,alcol1_0.99,alcol2_0.01,alcol2_0.1,alcol2_0.5,alcol2_0.9,alcol2_0.99,alcol3_0.01,alcol3_0.1,alcol3_0.5,alcol3_0.9,alcol3_0.99,snack1_0.01,snack1_0.1,snack1_0.5,snack1_0.9,snack1_0.99,snack2_0.01,snack2_0.1,snack2_0.5,snack2_0.9,snack2_0.99,snack3_0.01,snack3_0.1,snack3_0.5,snack3_0.9,snack3_0.99,bento1_0.01,bento1_0.1,bento1_0.5,bento1_0.9,bento1_0.99,bento2_0.01,bento2_0.1,bento2_0.5,bento2_0.9,bento2_0.99,bento3_0.01,bento3_0.1,bento3_0.5,bento3_0.9,bento3_0.99,bento4_0.01,bento4_0.1,bento4_0.5,bento4_0.9,bento4_0.99,tild1_0.01,tild1_0.1,tild1_0.5,tild1_0.9,tild1_0.99,tild2_0.01,tild2_0.1,tild2_0.5,tild2_0.9,tild2_0.99,men1_0.01,men1_0.1,men1_0.5,men1_0.9,men1_0.99,men2_0.01,men2_0.1,men2_0.5,men2_0.9,men2_0.99,men3_0.01,men3_0.1,men3_0.5,men3_0.9,men3_0.99,men4_0.01,men4_0.1,men4_0.5,men4_0.9,men4_0.99,men5_0.01,men5_0.1,men5_0.5,men5_0.9,men5_0.99,men6_0.01,men6_0.1,men6_0.5,men6_0.9,men6_0.99
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,16.0,20.0,21.0,22.0,52.0,33.0,60.0,63.0,64.0,88.0,24.0,26.0,30.0,34.0,40.0,6.0,27.0,40.0,51.0,93.0,20.0,38.0,50.0,64.0,92.0,53.0,87.0,96.0,138.0,188.0,24.0,46.0,58.0,70.0,146.0,109.0,135.0,153.0,180.0,208.0,191.0,210.0,224.0,248.0,263.0,190.0,215.0,235.0,255.0,271.0,19.0,43.0,51.0,53.0,55.0,36.0,60.0,74.0,76.0,84.0,6.0,47.0,53.0,60.0,58.0,40.0,56.0,62.0,63.0,67.0,37.0,59.0,70.0,79.0,81.0,16.0,36.0,39.0,39.0,55.0,18.0,30.0,35.0,39.0,55.0,10.0,20.0,22.0,22.0,41.0,27.0,37.0,42.0,45.0,50.0,22.0,23.0,24.0,26.0,27.0,7.0,8.0,8.0,9.0,16.0,52.0,56.0,61.0,62.0,89.0,41.0,44.0,51.0,52.0,127.0,45.0,53.0,56.0,61.0,111.0,18.0,23.0,26.0,29.0,33.0,10.0,17.0,18.0,21.0,29.0,27.0,35.0,43.0,46.0,58.0,25.0,46.0,51.0,53.0,59.0,23.0,37.0,44.0,46.0,54.0,35.0,63.0,72.0,78.0,79.0,10.0,26.0,28.0,30.0,33.0,4.0,11.0,12.0,18.0,20.0,8.0,9.0,11.0,13.0,17.0,30.0,44.0,52.0,60.0,64.0,21.0,27.0,31.0,34.0,35.0,21.0,32.0,36.0,43.0,52.0,26.0,35.0,38.0,48.0,54.0,17.0,35.0,39.0,46.0,49.0,21.0,34.0,38.0,46.0,66.0
2,16.0,18.0,20.0,21.0,40.0,34.0,53.0,56.0,56.0,86.0,24.0,26.0,30.0,33.0,38.0,7.0,18.0,25.0,49.0,86.0,27.0,35.0,32.0,65.0,99.0,63.0,78.0,77.0,125.0,190.0,24.0,51.0,48.0,61.0,146.0,106.0,133.0,147.0,166.0,213.0,189.0,202.0,228.0,240.0,252.0,187.0,203.0,227.0,240.0,268.0,19.0,25.0,30.0,34.0,36.0,36.0,45.0,53.0,58.0,71.0,5.0,7.0,11.0,14.0,24.0,26.0,28.0,32.0,37.0,57.0,32.0,34.0,38.0,50.0,57.0,17.0,24.0,26.0,32.0,65.0,18.0,26.0,27.0,35.0,53.0,10.0,16.0,19.0,25.0,37.0,27.0,32.0,34.0,36.0,49.0,22.0,23.0,25.0,26.0,27.0,7.0,7.0,8.0,9.0,16.0,64.0,68.0,72.0,76.0,91.0,46.0,58.0,65.0,71.0,127.0,50.0,64.0,72.0,80.0,111.0,19.0,28.0,32.0,39.0,39.0,9.0,28.0,33.0,35.0,36.0,27.0,41.0,53.0,59.0,68.0,8

In [278]:
df_ans.to_csv("../work/submission/sub_lgb_lag_first_valid3_and_lgbbase_ansamble.csv")

In [274]:
# # 目視確認
# pred_median_col = [c for c in submit_df.columns if '_0.5' in c]
# test_add_pred = test_df.merge(submit_df[pred_median_col].reset_index(), on='id', how='left')
# test_add_pred.columns = [c.replace('_0.5', '') if '_0.5' in c else c for c in test_add_pred.columns]
# test_add_pred[target_columns] = test_add_pred[target_columns].astype(float)
# test_add_pred.head(3)

In [275]:
# # 移動平均
# window=7
# train_rolling = train_df.rolling(window, min_periods=1).mean()
# test_rolling = test_add_pred.rolling(window, min_periods=1).mean()
# all_rolling = pd.concat([train_df, test_add_pred], axis=0).reset_index(drop=True).rolling(window, min_periods=1).mean()

In [276]:
# # 目視確認
# plot_col = [c for c in train_df.columns if c not in ['id', 'date', 'time', 'year', 'month', 'day', 'weekday', 'day_of_year']]
# ncols = len(plot_col) // 13
# plt.subplots(14, ncols, sharey=True, sharex=True, figsize=(30, 80))
# for i, col in enumerate(plot_col):
#     plt.subplot(14, ncols, i+1)
#     plt.plot(train_rolling.index[window:], train_rolling[col][window:], alpha=1, color='blue', label='train')
#     plt.plot(all_rolling.index[-len(test_rolling):], all_rolling[col][-len(test_rolling):], alpha=1, color='red', label='test')
#     for x in [20,51,81,112,143,173.204,234,265,296,324,350]:
#         plt.axvline(x)
#     plt.xlabel(col)
#     plt.legend()
#     plt.xticks([])
# plt.show()